In [1]:
%pip install optuna
%pip install lightgbm


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import optuna
from catboost import CatBoostClassifier
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as mcc

In [15]:
train = pd.read_csv('data/cleanedtrain.csv')
test = pd.read_csv('data/cleanedtest.csv')

In [16]:
train.set_index("id", drop=True, inplace=True)
train.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,,
0,e,8.80,f,s,u,f,a,c,w,4.51,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,p,4.51,x,h,o,f,a,c,n,4.79,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,e,6.94,f,s,b,f,x,c,w,6.85,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,e,3.88,f,y,g,f,s,NaN,g,4.16,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,e,5.85,x,l,w,f,d,NaN,w,3.37,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [17]:
y = train['class']
X = train.drop('class', axis = 1)

In [18]:
y.head()
X.head()

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,
0,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [33]:
features = X.columns.tolist()
print(features)

['cap-diameter', 'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']


In [20]:
def tostring(column,df):
    df[column] = df[column].astype(str)

In [21]:
#Optuna
#study = optuna.create_study(direction="maximize")


In [29]:

for feat in features:
    X[feat] = X[feat].astype('category')




In [30]:

def mymcc(y_true, y_pred):
    tp = np.float64(np.sum(y_true & y_pred))
    fn = np.float64(np.sum(y_true & np.logical_not(y_pred)))
    fp = np.float64(np.sum(np.logical_not(y_true) & y_pred))
    tn = np.float64(np.sum(np.logical_not(y_true) & np.logical_not(y_pred)))
    denom = (tp+fn)*(fp+tn)*(tp+fp)*(fn+tn)
    if denom == 0:
        return 0
    else:
        return (tp*tn-fn*fp)/np.sqrt(denom)

In [31]:
def objective(trial,X,y,model_name):

    
    if model_name == 'lightgbm':
        params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 10000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 10000, step=100),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 10.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 10.0),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.0, 15.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.95),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.95),
        "objective": "binary",
        "random_state": 0
    }
        model = lgb.LGBMClassifier(**params)

    elif model_name == 'catboost':
        params = {
            'iterations': trial.suggest_int('iterations', 12, 14000),  # boost trees
            'depth': trial.suggest_int('depth', 6, 15),  # Depth of the trees
            'learning_rate': trial.suggest_float('learning_rate', 0.002, 0.3),  # Step size for gradient updates
            'random_strength': trial.suggest_int('random_strength', 1, 10),  # variability
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),  # randomness in bagging
            'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),  # Overfitting detector type
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10.0),  # L2 regularization coefficient for leaves
            'border_count': trial.suggest_int('border_count', 32, 255),  # Number of splits in numerical features
            'verbose': 100,  # Output verbosity
            'loss_function': 'Logloss',  # Loss function for binary classification
            'eval_metric': 'MCC',  # Evaluation metric for optimization (Matthews Correlation Coefficient)
            'random_seed': 0,  # Seed for random number generator
        }
        model = CatBoostClassifier(**params)

    stratkfold = StratifiedKFold(n_splits = 5, random_state = 0, shuffle = True)
    mcc_scores = []

    for train_i, test_i in stratkfold.split(X,y):
        x_train, x_test = X.iloc[train_i], X.iloc[test_i]
        y_train, y_test = y.iloc[train_i], y.iloc[test_i]
        if model_name == 'lightgbm':
            model.fit(x_train, y_train, eval_set=[(x_test, y_test)], callbacks=[
                    lgb.early_stopping(stopping_rounds=100, verbose=100),
                    lgb.log_evaluation(period=0)
                ],eval_metric = 'mcc')
        else:
            model.fit(x_train, y_train, eval_set=[(x_test, y_test)], early_stopping_rounds=100, verbose=100,cat_features =features )
        preds = model.predict(x_test)
        mcc = mymcc(y_test, preds)
        mcc_scores.append(mcc)
        
    return  np.mean(mcc_scores)



In [32]:
 study = optuna.create_study(direction='maximize')

func = lambda trial: objective(trial,X= X,y = y,model_name = 'catboost')
study.optimize(func, n_trials=500)
print('Best trial:', study.best_trial.params)

[I 2024-08-22 16:27:54,918] A new study created in memory with name: no-name-71d5f574-4f21-4606-ab93-d88881edc85b


0:	learn: 0.9441069	test: 0.9474645	best: 0.9474645 (0)	total: 1.14s	remaining: 1h 55m 43s


[W 2024-08-22 16:28:41,192] Trial 0 failed with parameters: {'iterations': 6093, 'depth': 8, 'learning_rate': 0.21687346913995995, 'random_strength': 1, 'bagging_temperature': 0.3569784568767618, 'od_type': 'IncToDec', 'l2_leaf_reg': 7.359395189527159, 'border_count': 180} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/n3/_xbw88791wx3w7f1hz1vjvnc0000gn/T/ipykernel_75982/2651137622.py", line 3, in <lambda>
    func = lambda trial: objective(trial,X= X,y = y,model_name = 'catboost')
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/n3/_xbw88791wx3w7f1hz1vjvnc0000gn/T/ipykernel_75982/2923341208.py", line 51, in objective
    model.fit(x_train, y_train, eval_set=[(x_test, y_t

KeyboardInterrupt: 

In [1]:
#best params I 2024-08-24 07:58:09,986] Trial 0 finished with value: 0.9845525967812379 and parameters: {'iterations': 3979, 'depth': 14, 'learning_rate': 0.04794429552561989, 'random_strength': 5, 'bagging_temperature': 0.6839643118243162, 'od_type': 'IncToDec', 'l2_leaf_reg': 8.112600124584425, 'border_count': 247}. Best is trial 0 with value: 0.9845525967812379.
#bottlenecked by kaggle runtime of 12 hours limit... could only get 10 trails. 